# Introduction to TMS and Motor Thresholds

Authored by Remy Cohan [neuropsis.org] with minor edits (mostly formatting) by Peter Kohler [kohlerlab.com].

As discussed during the lecture and in the video played during class, in TMS research we often measure motor thresholds (MTs) using single-pulse TMS to primary motor cortex (M1). The resting motor threshold (rMT) is defined as the minimum TMS intensity that produces an electromyographic (EMG) response of 50 µV in a relaxed muscle. MTs are important because they can reflect changes in neural excitability and plasticity.

On the other hand, repetitive TMS (rTMS) delivers multiple pulses in patterns (e.g., 1 Hz, 10 Hz, or theta burst stimulation). rTMS is gaining popularity as a potential treatment for neuropsychiatric conditions and in neurorehabilitation.

To study the effects of rTMS, we can measure MTs using single-pulse TMS before and after an rTMS protocol. This allows us to see how the rTMS protocol changes neural activity in a specific brain area, which is critical when designing treatment protocols.

Remember the distinction:

Single-pulse TMS: Measures MTs and neural excitability in primary motor cortex (M1).

Repetitive TMS (rTMS): Modulates neural activity to induce plasticity.

In the data below, pre-rTMS refers to a baseline single-pulse TMS measure of MTs before the rTMS protocol. Post-rTMS refers to the same measure after the rTMS protocol. Also, under the Condition column you have a (for active/real TMS) and s (for sham TMS), and under the Sex column f (for female) and m (for male). 

This background should help you interpret the data and answer the following questions.

It is a good idea to review some of Pandas functions such as groupby and .agg here: https://github.com/marsja/jupyter/blob/master/pandas_groupby_tutorial.ipynb

One more thing: in real TMS datasets, before you interpret anything, you do quick sanity checks (missing values, duplicates, and obviously weird threshold values). That is what we are doing here. This is neuroscience, not a pure coding exercise.

## Preamble: Import some packages

As is often the case, we start by importing some Python modules.

Here we import pandas for data handling and scipy for correlation analysis. We also import modules such as matplotlib and seaborn for plotting and numpy for data wrangling...don't say numpee (You can annoy a lot of ppl including me, Just kidding [half]); it's numpie, anything with py in this context refers to python (piethaan or piethin if you're a Brit)...RC.

In [ ]:
# install tools for getting data and set datapath
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    import shutil
    # download github repository to get access to the tms data
    shutil.rmtree("NRSC2200", ignore_errors=True)
    !git clone https://github.com/pjkohler/NRSC2200
    datapath = "NRSC2200/tms/tms_cohan_data.csv"
else:
    datapath = "tms_cohan_data.csv"

# import python modules to use for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

## Question 1: Data inspection and basic TMS sanity checks (1 point)

How many pre and post data points are present in the dataset?

To answer this, load the dataset and count the number of available data points for the pre_rMT and post_rMT columns.

Before you interpret anything in a TMS threshold dataset, do quick checks:

1) Are the columns you need actually there?
2) Are there obviously impossible or suspicious rMT values?
3) Do you have duplicate participant rows?

For this homework, treat rMT values < 20 or > 95 as suspicious (this is not a law of nature but true to a certain extent; I don't belive these values if I see them in a dataset!), it is just a reasonable QC flag for a teaching purposes!).

Please include only the lines of code that you changed in your submitted answer.

In [ ]:
# load the dataset
data = pd.read_csv(datapath)

# Remy Feb 2026: make condition labels robust (prevents dumb KeyErrors later)
data['condition'] = data['condition'].astype(str).str.strip().str.lower()

# Remy Feb 2026: make participant_id robust for duplicate checks
data['participant_id'] = data['participant_id'].astype(str)

# we give you this one
print("column names:", data.columns.tolist())

# Remy Feb 2026: basic required column check (if this fails, nothing else matters)
required_cols = ['participant_id', 'condition', 'sex', 'age', 'pre_rMT', 'post_rMT']
missing_required = [c for c in required_cols if c not in data.columns]
print("missing required columns:", missing_required)

# Remy Feb 2026: a quick look at labels
print("conditions present:", sorted(data['condition'].dropna().unique().tolist()))  ### Students fill this: sorted(data['condition'].dropna().unique().tolist()))
print("sex labels present:", sorted(data['sex'].dropna().astype(str).str.strip().str.lower().unique().tolist()))  ### Students fill this: sorted(data['sex'].dropna().astype(str).str.strip().str.lower().unique().tolist()))

In [ ]:
# Using the variable names pre_count and post_count, count the number of pre and post rMT data points and print the answer
# Hint: Use .notna() to check for missing values before counting.

# fill in the function to count non null values for pre_rMT
pre_count = int(data['pre_rMT'].notna().sum())  ### Students fill this: pre_rMT

# fill in the function to count non null values for post_rMT
post_count = int(data['post_rMT'].notna().sum())  ### Students fill this: post_rMT

print(f"number of Pre rMT data points: {pre_count}")
print(f"number of Post rMT data points: {post_count}")

# Remy Feb 2026: suspicious value QC flags (rMT < 20 or > 95)
n_suspicious_pre = int(((data['pre_rMT'] < 20) | (data['pre_rMT'] > 95)).sum())  ### Students fill this: ['pre_rMT'] < 20, data['pre_rMT'] ; ['pre_rMT'] > 95
n_suspicious_post = int(((data['post_rMT'] < 20) | (data['post_rMT'] > 95)).sum())  ### Students fill this: post_rMT and post_rMT 

print("suspicious pre_rMT values (<20 or >95):", n_suspicious_pre)
print("suspicious post_rMT values (<20 or >95):", n_suspicious_post)

# Remy Feb 2026: duplicate participant rows check (participants appearing more than once)
dup_participants = data['participant_id'].value_counts().loc[lambda s: s > 1]  ### Students fill this: participant_id
print("participants appearing more than once:")
print(dup_participants)

# Remy Feb 2026: quick numeric signature
sig_q1 = int(pre_count * 3 + post_count * 5 + n_suspicious_pre * 7 + n_suspicious_post * 11)
print("Q1 signature:", sig_q1)

## Question 2: Check for missing values (1 point)

Missing values are a big deal in TMS threshold work (coil placement drift, EMG noise, participant discomfort, etc.).

1) Report missing values for each column.
2) Then answer this (choose one letter):

Which is more consistent with missing rMT values in a motor threshold dataset?

A) technical/acquisition issue (e.g., EMG noise, poor relaxation, coil positioning)
B) missing values are expected and do not matter
C) missing values prove the rTMS protocol worked

Please include only the lines of code that you changed in your submitted answer (and the letter you chose).

In [ ]:
# fill in the function to check for missing values in the data
# hint: The .isnull().sum() method shows missing values for each column.

missing_values = data.isnull().sum()  ### Students fill this: data.isnull().sum()

print("missing values in each column:")
print(missing_values)

# Remy Feb 2026: which column has the most missing values?
most_missing_col = missing_values.idxmax()  ### Students fill this: missing_values.idxmax()
most_missing_n = int(missing_values.max())  ### Students fill this: int(missing_values.max()
print("most missing column:", most_missing_col)
print("missing count:", most_missing_n)

# Remy Feb 2026: quick numeric signature
sig_q2 = int(most_missing_n * 13 + len(missing_values) * 17)
print("Q2 signature:", sig_q2)

## Question 3: Participant demographics and threshold characterisation (2 points)

1) How many males and females are there in the dataset?

2) What are the mean and standard deviation of age, pre-rMT, and post-rMT values separately for Active vs Sham?

3) Compute the mean change score (post minus pre) for Active and Sham. In TMS terms, a decrease in rMT means less stimulator output is needed to trigger a response (higher excitability).

Your outputs must be printed exactly by the code: sex_counts, stats_summary, and delta_summary.

In [ ]:
# add code to count males and females
# hint: The .value_counts() function helps count categorical variables.

sex_counts = data['sex'].astype(str).str.strip().str.lower().value_counts()  ### Students fill this: sex and value_counts() 

print("count of participants by sex:")
print(sex_counts)

# compute mean and standard deviation for age, pre-rMT, and post-rMT values by condition
# hint: Use .groupby() to separate data by condition before applying statistics.
# Remy Feb 2026: .agg is cleaner than calling mean/std repeatedly.

stats_summary = data.groupby('condition')[['age', 'pre_rMT', 'post_rMT']].agg(['mean', 'std']).round(3)  ### Students fill this: ['age', 'pre_rMT', 'post_rMT']].agg(['mean', 'std']

print("mean and standard deviation for Age, Pre MTs, and Post MTs by condition:")
print(stats_summary)

# Remy Feb 2026: change score (post - pre) by condition
data['delta_rMT'] = data['post_rMT'] - data['pre_rMT']
delta_summary = data.groupby('condition')[['delta_rMT']].agg(['mean', 'std']).round(3)  ### Students fill this: data.groupby('condition')[['delta_rMT']]
print("mean and standard deviation for ΔrMT (post - pre) by condition:")
print(delta_summary)

# Remy Feb 2026: summary: what % decreased rMT (delta < 0)?
responder_rate = (data['delta_rMT'] < 0).groupby(data['condition']).mean().mul(100).round(1)  ### Students fill this: ['delta_rMT'] < 0
print("% with ΔrMT < 0 (by condition):")
print(responder_rate)

# Remy Feb 2026: quick numeric signature
sig_q3 = round(float(delta_summary.select_dtypes(include=[np.number]).values.sum()), 3)
print("Q3 signature:", sig_q3)

## Question 4: Pre vs post rMT values by condition (1 point)

1) Create the boxplot as before.

2) Create a within-participant line plot where each participant has a line from pre to post, separated by condition.

3) Print the mean change (post minus pre) for Active and Sham (use delta_rMT from Q3).

4) Print the % of participants with ΔrMT < 0 for Active vs Sham (reuse your responder summary from Q3).

In [ ]:
# create boxplot comparing pre and post rMT values by condition
# hint: Convert data into long format using pd.melt() before plotting.

tms_data = pd.melt(
    data,
    id_vars=['participant_id', 'condition'],  ### Students fill this: 'participant_id', 'condition'
    value_vars=['pre_rMT', 'post_rMT'],  ### Students fill this: 'pre_rMT', 'post_rMT'
    var_name='time',
    value_name='rMT'
)

plt.figure(figsize=(10, 6))
sns.boxplot(x='condition', y='rMT', hue='time', data=tms_data, palette='Set2')
plt.title("Pre vs Post rMT by Condition")
plt.xlabel("Condition")
plt.ylabel("rMT (TMS dose)")
plt.legend(title="Time")
plt.show()

# Remy Feb 2026: within participant change plot (each line = one participant)
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=tms_data,
    x='time',
    y='rMT',
    hue='condition',
    units='participant_id',
    estimator=None,
    alpha=0.35
)
plt.title("Within participant pre to post rMT (each line = one participant)")
plt.xlabel("Time")
plt.ylabel("rMT (TMS dose)")
plt.show()

# print mean change by condition (post - pre)
mean_change = data.groupby('condition')['delta_rMT'].mean().round(3)  ### Students fill this: 'delta_rMT'
print("Mean change (post - pre) by condition:")
print(mean_change)

# reprint responder summary to keep this question self contained, I'm so nice!
responder_rate = (data['delta_rMT'] < 0).groupby(data['condition']).mean().mul(100).round(1)  ### Students fill this: 'condition'
print("% with ΔrMT < 0 (by condition):")
print(responder_rate)

# Remy Feb 2026: rule based conclusion label
diff_change = float(mean_change.loc['a'] - mean_change.loc['s']) if ('a' in mean_change.index and 's' in mean_change.index) else np.nan
if np.isfinite(diff_change) and diff_change <= -1.0:
    label = 'Active decreases more'
elif np.isfinite(diff_change) and diff_change >= 1.0:
    label = 'Active increases more'
else:
    label = 'Similar'
print("Q4 conclusion label:", label)

# Remy Feb 2026: quick numeric signature
sig_q4 = round(float(mean_change.sum()), 3) if hasattr(mean_change, 'sum') else None
print("Q4 signature:", sig_q4)

## Question 5: Correlation between age and baseline rMT (2 points)

Run the code below to plot age against baseline rMT.

Your tasks:

1) Report r (Pearson correlation) and p-value.

2) Categorise the strength of the relationship using this rule:

Weak: |r| < 0.30
Moderate: 0.30 ≤ |r| < 0.50
Strong: |r| ≥ 0.50

3) Choose ONE of the following (A/B/C):

A) older age is associated with higher baseline rMT
B) older age is associated with lower baseline rMT
C) no clear association (if your relationship is Weak)

In [ ]:
# correlation plot for age vs pre rMTs
corr, pval = pearsonr(data['age'], data['pre_rMT'])  ### Students fill this: 'age' and 'pre_rMT' 

plt.figure(figsize=(8, 6))
sns.regplot(x='age', y='pre_rMT', data=data, scatter_kws={'s': 50, 'alpha': 0.7, 'clip_on': False}, color='green')
plt.title("age vs baseline rMT")
plt.xlabel("age (years)")
plt.ylabel("baseline rMT")
plt.xlim(20, 60)
plt.ylim(45, 75)
plt.text(35, 50, f"correlation between age and pre rMT:\nr = {corr:.2f}", fontsize=12)
plt.show()

# Remy Feb 2026: print r and p-value
print("Pearson r:", round(corr, 3))
print("p-value:", round(pval, 4))

# Remy Feb 2026: strength category (rule-based)
abs_r = abs(corr)
if abs_r < 0.30:
    strength = 'Weak'
elif abs_r < 0.50:
    strength = 'Moderate'
else:
    strength = 'Strong'
print("Strength category:", strength)

# Remy Feb 2026: direction helper
direction = 'positive' if corr > 0 else 'negative'
print("Direction (sign of r):", direction)

# Remy Feb 2026: quick numeric signature
sig_q5 = round(float(corr * 1000 + pval * 10000), 2)
print("Q5 signature:", sig_q5)

## Question 6: Interpretation (2 points)

Use your outputs from Q3/Q4 (mean change post minus pre by condition and % with ΔrMT < 0) and Q5 (correlation strength category).

Answer using this exact format:

1) rTMS effect on excitability (choose one):

A) If rMT decreases post-rTMS, excitability increased (less intensity needed)
B) If rMT decreases post-rTMS, excitability decreased
C) rMT cannot tell you anything about excitability

2) Age statement (choose one):

D) Age likely matters for baseline rMT in this dataset (Moderate/Strong)
E) Age likely does not matter much for baseline rMT in this dataset (Weak)

3) One sentence only that uses your numbers (ΔActive, ΔSham, % with Δ<0 for each condition, and your strength category).